In [1]:
import pickle
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, Model
import numpy as np

2024-12-07 16:35:05.060783: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 16:35:05.113420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 16:35:05.113474: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 16:35:05.113523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 16:35:05.125429: I tensorflow/core/platform/cpu_feature_g

In [3]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
tf.config.set_visible_devices([], 'GPU')

In [4]:
pickle_MCI_Features_file_path = r'./feature_extraction/MCI_func_features.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_funct_features = pickle.load(file)
MCI_funct_features.shape
MCI_funct_features = np.array(MCI_funct_features)

In [5]:
MCI_funct_features.shape

(52, 13456)

In [6]:
pickle_MCI_Features_file_path = r'feature_extraction/cat12_MCI_structural_features.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_Struct_features = pickle.load(file)
MCI_Struct_features.shape
MCI_Struct_features = np.array(MCI_Struct_features)

In [7]:
MCI_Struct_features.shape

(52, 169, 205, 169)

In [8]:
# Define the same file path
file_path = r'feature_extraction/cat12_AD_structural_features.pkl'

# Load the NumPy array from the file
with open(file_path, "rb") as file:  # 'rb' means read binary
    AD_struct_features = pickle.load(file)

print("structural_features loaded successfully!")
print("Shape of loaded array:", AD_struct_features.shape)


structural_features loaded successfully!
Shape of loaded array: (40, 169, 205, 169)


In [9]:
pickle_AD_Features_file_path = r"feature_extraction/AD_func_features.pkl"
with open(pickle_AD_Features_file_path, 'rb') as file:
    AD_funct_features = pickle.load(file)
AD_funct_features = np.array(AD_funct_features)
AD_funct_features.shape

(40, 13456)

In [10]:
# Define the same file path
file_path = r'feature_extraction/cat12_CN_structural_features.pkl'

# Load the NumPy array from the file
with open(file_path, "rb") as file:  # 'rb' means read binary
    CN_struct_features = pickle.load(file)

print("structural_features loaded successfully!")
print("Shape of loaded array:", CN_struct_features.shape)

structural_features loaded successfully!
Shape of loaded array: (30, 169, 205, 169)


In [11]:
pickle_AD_Features_file_path = r"feature_extraction/CN_func_features.pkl"
with open(pickle_AD_Features_file_path, 'rb') as file:
    CN_funct_features = pickle.load(file)
CN_funct_features.shape

(30, 13456)

In [12]:
MCI_funct_features.shape, MCI_Struct_features.shape

((52, 13456), (52, 169, 205, 169))

In [13]:
AD_funct_features.shape, AD_struct_features.shape

((40, 13456), (40, 169, 205, 169))

In [14]:
CN_funct_features.shape, CN_struct_features.shape

((30, 13456), (30, 169, 205, 169))

In [15]:

# Data Shapes
AD_Struct_features = AD_struct_features
# Labels
MCI_labels = np.zeros(MCI_funct_features.shape[0])  # Label 0 for MCI
AD_labels = np.ones(AD_funct_features.shape[0])    # Label 1 for AD
CN_labels = np.array([2]*CN_funct_features.shape[0])

# Combine functional and structural features
funct_features = np.vstack((MCI_funct_features, AD_funct_features, CN_funct_features))
struct_features = np.vstack((MCI_Struct_features, AD_Struct_features, CN_struct_features))
labels = np.hstack((MCI_labels, AD_labels, CN_labels))

In [17]:
struct_features = np.expand_dims(struct_features, axis=-1)

In [16]:
funct_features = funct_features.reshape(-1, 116, 116)
funct_features = np.expand_dims(funct_features, axis=-1)
funct_features.shape

(122, 116, 116, 1)

In [ ]:
funct_features = np.random.rand(15, 116, 116, 1)
struct_features = np.random.rand(15, 169, 205, 169)
labels = np.array([0]*5 + [1]*5 + [2]*5) 

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
# Split data into training and testing sets
X_train_3d, X_test_3d, X_train_2d, X_test_2d, y_train, y_test = train_test_split(
    struct_features,
    funct_features,
    labels,
    test_size=0.2,
    random_state=42
)

In [58]:
# Discriminator labels (real images are labeled as 1)
real_labels_train = np.ones((X_train_3d.shape[0], 1))
real_labels_test = np.ones((X_test_3d.shape[0], 1))

# For simplicity, use real labels for reconstructed images
disc_targets_3d_train = real_labels_train
disc_targets_2d_train = real_labels_train

disc_targets_3d_test = real_labels_test
disc_targets_2d_test = real_labels_test

In [67]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def build_encoder_3d_2d(input_shape):
    inputs = Input(shape=input_shape) # 169x205 x169 (input, 169 channels)
    x = layers.Conv2D(32, (3, 3), activation='gelu', padding='valid')(inputs) # 166×203 ×32
    x = layers.MaxPooling2D((2, 2))(x) # 83×101×83 x32
    x = layers.Conv2D(64, (3, 3), activation='gelu', padding='valid')(x) # 81×99 x64
    x = layers.MaxPooling2D((2, 2))(x) # 40×49×40 x64
    x = layers.Conv2D(128, (3, 3), activation='gelu', padding='valid')(x) # 38×47 x128
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    encoder = Model(inputs, latent, name='encoder_3d')
    return encoder

In [68]:
def build_encoder_2d(input_shape):
    inputs = Input(shape=input_shape) # 116x116 x1
    x = layers.Conv2D(32, (3, 3), activation='gelu', padding='valid')(inputs) # 114x114 x32
    x = layers.MaxPooling2D((2, 2))(x) # 57x57 x32
    x = layers.Conv2D(64, (3, 3), activation='gelu', padding='valid')(x) # 55x55 x64
    x = layers.MaxPooling2D((2, 2))(x) # 27x27 x64
    x = layers.Conv2D(128, (3, 3), activation='gelu', padding='valid')(x) # 25x25 x128
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    encoder = Model(inputs, latent, name='encoder_2d')
    return encoder

In [69]:
def build_decoder_3d_2d(latent_dim, output_shape):
    # output_shape: (height, width, channels) - Original 2D data shape with high number of channels
    height, width, channels = output_shape[0], output_shape[1], output_shape[2]
    
    latent_inputs = Input(shape=(latent_dim,))
    
    # Define the dimensions to reshape the latent vector
    reshape_height = height // 8
    reshape_width = width // 8
    reshape_filters = 128  # Number of filters
    
    x = layers.Dense(reshape_height * reshape_width * reshape_filters, activation='gelu')(latent_inputs)
    x = layers.Reshape((reshape_height, reshape_width, reshape_filters))(x)
    
    # Upsample spatial dimensions
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='gelu')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='gelu')(x)
    x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='gelu')(x)
    
    # Final layer to match the number of channels
    x = layers.ZeroPadding2D(padding=((0, 1), (0, 5)))(x)
    outputs = layers.Conv2DTranspose(channels, (3, 3), padding='same', activation='sigmoid')(x)
    assert outputs.shape[1:] == output_shape, f"Expected output shape {output_shape} but got {outputs.shape[1:]}"
    decoder = Model(latent_inputs, outputs, name='decoder_3d')
    return decoder

In [70]:
def build_decoder_2d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    x = layers.Dense((output_shape[0]//4)*(output_shape[1]//4)*64, activation='gelu')(latent_inputs)
    x = layers.Reshape((output_shape[0]//4, output_shape[1]//4, 64))(x)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(1, (3, 3), padding='valid', activation='sigmoid')(x)
    outputs = layers.Cropping2D(cropping=((2, 3), (2, 3)))(x) 

    assert outputs.shape[1:-1] == output_shape, f"Output shape {outputs.shape[1:-1]} does not match expected shape {output_shape}"


    decoder = Model(latent_inputs, outputs, name='decoder_2d')
    return decoder

In [71]:
def build_discriminator_3d_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_3d')
    return discriminator

In [72]:
def build_discriminator_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_2d')
    return discriminator

In [73]:
def build_classifier(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    # x = layers.Dense(256, activation='gelu')(inputs)
    x = layers.Dense(128, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    classifier = Model(inputs, outputs, name='classifier')
    return classifier

In [74]:
# Input shapes
input_shape_3d = (169, 205, 169)
input_shape_2d = (116, 116, 1)
latent_dim = 256
num_classes = 3

# Build models
encoder_3d = build_encoder_3d_2d(input_shape_3d)
encoder_2d = build_encoder_2d(input_shape_2d)
decoder_3d = build_decoder_3d_2d(latent_dim, input_shape_3d)
decoder_2d = build_decoder_2d(latent_dim, input_shape_2d[:-1])
discriminator_3d = build_discriminator_3d_2d(input_shape_3d)
discriminator_2d = build_discriminator_2d(input_shape_2d)
# Build classifier that accepts concatenated latent vectors
classifier = build_classifier(latent_dim * 2, num_classes)

# Inputs
input_3d = Input(shape=input_shape_3d)
input_2d = Input(shape=input_shape_2d)

# Encoding
latent_3d = encoder_3d(input_3d)
latent_2d = encoder_2d(input_2d)

# Concatenate the two latent representations
combined_latent = layers.Concatenate()([latent_3d, latent_2d])

# Decoding
reconstructed_3d = decoder_3d(latent_3d)
reconstructed_2d = decoder_2d(latent_2d)

# Discriminator outputs
disc_output_3d = discriminator_3d(reconstructed_3d)
disc_output_2d = discriminator_2d(reconstructed_2d)

# Get classification output
classification_output = classifier(combined_latent)

# Define the combined model
model = Model(
    inputs=[input_3d, input_2d],
    outputs=[
        reconstructed_3d,
        reconstructed_2d,
        disc_output_3d,
        disc_output_2d,
        classification_output
    ]
)

In [ ]:
# Loss weights
lambda_reconstruction = 1.0
lambda_adversarial = 0.1
lambda_classification = 1.0

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    loss={
        'decoder_3d': 'mse',
        'decoder_2d': 'mse',
        'discriminator_3d': tf.keras.losses.BinaryCrossentropy(),
        'discriminator_2d': tf.keras.losses.BinaryCrossentropy(),
        'classifier': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'decoder_3d': lambda_reconstruction,
        'decoder_2d': lambda_reconstruction,
        'discriminator_3d': lambda_adversarial,
        'discriminator_2d': lambda_adversarial,
        'classifier': lambda_classification
    },
    metrics={
        'classifier': 'accuracy'
    }
)

In [76]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


# Discriminator labels (real images are labeled as 1)
real_labels = np.ones((struct_features.shape[0], 1))
fake_labels = np.zeros((struct_features.shape[0], 1))

# For simplicity, use real labels for reconstructed images (you can adjust as needed)
disc_targets_3d = real_labels
disc_targets_2d = real_labels


# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}


In [79]:
# Train the model
history = model.fit(
    [X_train_3d, X_train_2d],
    {
        'decoder_3d': X_train_3d,
        'decoder_2d': X_train_2d,
        'discriminator_3d': disc_targets_3d_train,
        'discriminator_2d': disc_targets_2d_train,
        'classifier': y_train
    },
    epochs=20,
    batch_size=8,
    validation_data=(
        [X_test_3d, X_test_2d],
        {
            'decoder_3d': X_test_3d,
            'decoder_2d': X_test_2d,
            'discriminator_3d': disc_targets_3d_test,
            'discriminator_2d': disc_targets_2d_test,
            'classifier': y_test
        }
    ),
    # class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/20


ValueError: in user code:

    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(



In [31]:
# Evaluate the classifier accuracy on the test set
evaluation = model.evaluate(
    [X_test_3d, X_test_2d],
    {
        'decoder_3d': X_test_3d,
        'decoder_2d': X_test_2d,
        'discriminator_3d': disc_targets_3d_test,
        'discriminator_2d': disc_targets_2d_test,
        'classifier': y_test
    },
    verbose=0
)

# Get the index of the classification loss and accuracy (depends on the order of outputs)
classification_accuracy = evaluation[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Test Accuracy: {classification_accuracy * 100:.2f}%')

Classifier Test Accuracy: 56.00%


In [32]:
evaluation_train = model.evaluate(
    [X_train_3d, X_train_2d],
    {
        'decoder_3d': X_train_3d,
        'decoder_2d': X_train_2d,
        'discriminator_3d': disc_targets_3d_train,
        'discriminator_2d': disc_targets_2d_train,
        'classifier': y_train
    },
    verbose=0
)

classification_accuracy = evaluation_train[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Train Accuracy : {classification_accuracy * 100:.2f}')

Classifier Train Accuracy : 100.00


In [36]:
model.save("models/fusion_model_1.h5")

/home/tripti/miniconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


OSError: [Errno 28] Can't close file (file write failed: time = Thu Nov 28 18:09:47 2024
, filename = 'models/fusion_model_1.h5', file descriptor = 84, errno = 28, error message = 'No space left on device', buf = 0xbfd3120, total write size = 47824, bytes this sub-write = 47824, bytes actually written = 18446744073709551615, offset = 0)

In [33]:
X_train_3d.shape, X_test_2d.shape

((97, 169, 205, 169), (25, 116, 116, 1))

In [ ]:
model.predict([X_test_2d, X_test_2d])

In [1]:
X_test_2d

NameError: name 'X_test_2d' is not defined

In [2]:
import pickle

In [ ]:
with open(r"./test.pkl", 'wb') as file:
    pickle.dump(X_test_2d, file)

In [ ]:
with open(r"./test.pkl", 'rb') as file:
    test = pickle.load(file)